# 페이스북 크롤링

In [1]:
# 페이지 스크롤 횟수 설정
num_scrolldown = 1

In [2]:
from bs4 import BeautifulSoup as bs
import time
import random
from datetime import datetime
from selenium import webdriver
# import getpass

In [3]:
import pandas as pd

## 크롤링할 사이트 접속

In [34]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(3)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)

## 로그인

In [53]:
# account_box = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[1]/label/input')
# password_box = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[2]/label/input')
# login_button = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[3]/div/div/div[1]/div/span/span')

# # id, password 입력 받기

# id = input("ID를 입력하세요 : ")
# password = getpass.getpass("password(비밀번호)를 입력하세요 : ")

# account_box.send_keys(id)
# time.sleep( random.uniform(0.5, 1) )
# password_box.send_keys(password)
# time.sleep( random.uniform(0.5, 1) )
# login_button.click()
# driver.implicitly_wait(3)

## 페이지 스크롤 다운

In [5]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3) )

## 크롤링

In [6]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print(soup.title.text)

Gyeonggi Korea | Facebook


# 게시글 크롤링

## 게시글 수집

In [33]:
# 게시글 개수
div = soup.find_all('div', 'du4w35lb l9j0dhe7')
print("수집한 게시글 수 : ", len(div) )

수집한 게시글 수 :  7


In [49]:
temp = soup.select('div.du4w35lb.l9j0dhe7 span.tojvnm2t.a6sixzi8.abs2jz4q.a8s20v7p.t1p8iaqh.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.iyyx5f41 a')
len(temp)

7

In [50]:
temp[0].attrs['aria-label']

'어제 오후 2:00'

In [51]:
temp[1].attrs['aria-label']

'3월 28일 오후 2:00'

## 날짜 수집

In [25]:
# 글 날짜 수집
num_date_list = []
for one in div:
    raw_date_info = one.find_all('div', 'qzhwtbm6 knvmm38d')[1]
    raw_date = raw_date_info.find('a').attrs['aria-label']

    # # 날짜 형식 변환해서 리스트로 저장
    if ('시간' in raw_date) or ('분' in raw_date):
        temp = datetime.today().strftime("%Y-%m-%d")
        num_date_list.append(temp)
    elif '어제' in raw_date:
        temp = datetime.today().strftime("%Y-%m-%d")
        temp = temp.split('-')
        year = temp[0]
        month = temp[1]
        day = int( temp[2] ) - 1
        temp_date = f'{year}-{month:0>2}-{day:0>2}'
        num_date_list.append(temp_date)
    else:
        temp = raw_date.split('일')[0]
        temp = temp.replace(' ', '')
        temp = temp.replace('년', '-')
        temp = temp.replace('월', '-')
        temp_li = temp.split('-')
        if len(temp_li) == 3 :
            year = temp_li[0]
            month = temp_li[1]
            day = temp_li[2]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
        elif len(temp_li) == 2:
            year = datetime.today().strftime("%Y")
            month = temp_li[0]
            day = temp_li[1]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
        else:
            temp_day = int(temp_li[0]) + 1
            temp = datetime.today().strftime("%Y-%m-%d")
            temp = temp.split('-')
            year = temp[0]
            month = temp[1]
            day = int( temp[2] ) - temp_day
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
            

print("변환한 날짜 리스트 원소수 : ", len(num_date_list) )

변환한 날짜 리스트 원소수 :  7


In [26]:
num_date_list

['2022-03-30',
 '2022-03-28',
 '2022-03-25',
 '2022-03-25',
 '2022-03-23',
 '2022-03-21',
 '2022-03-19']

## 제목 수집

In [99]:
# 글 제목 수집
title_list = []
for one in div:
    raw_title = one.find('div', 'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')
    if raw_title == None:
        raw_title = one.find('div', 'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql')
    title = raw_title.text
    title_list.append(title)

print("수집한 제목 수 : ", len(title_list))

수집한 제목 수 :  7


In [100]:
title_list[:5]

['어제 시험 감독관 일이 생각보다 힘들었다. 그곳에서만 1만보를 넘게 걸었다.발목도 좀 아프고 다리도 아프고 자고 일어나니 어깨도 아프다.',
 '오늘은 페이스북 날짜 크롤링 코드를 조금 손봤다.항상 수집한 날짜가 다른 데이터보다 1개씩 많아 처리를 한번 더 해야했는데, 수정해서 다른 데이터와 동일한 개수로 수집된다.',
 '오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자',
 '내 페이지의 정보를 내가 수집하면 차단되나?',
 '어떻게 내 페이지를 만드는 거지?어렵다']

## 조회수 수집

In [101]:
# 조회수
num_view_list = []
for one in div:
    num_view_list.append(0)

print("수집한 조회수 수 : ", len(num_view_list))

수집한 조회수 수 :  7


In [102]:
num_view_list[:5]

[0, 0, 0, 0, 0]

# 좋아요 수 수집

In [139]:
# 좋아요 수 수집
num_like_list = []
for one in div:
    raw_lsc_info = one.find('div', 'bp9cbjyn j83agx80 buofh1pr ni8dbmo4 stjgntxs')
    if raw_lsc_info == None:
        num_like_list.append(0)
    else:
        # 좋아요 수 수집
        raw_like_info = raw_lsc_info.find('span', 'np69z8it et4y5ytx j7g94pet b74d5cxt qw6c0r16 kb8x4rkr ed597pkb omcyoz59 goun2846 ccm00jje s44p3ltw mk2mc5f4 qxh1up0x qtyiw8t4 tpcyxxvw k0bpgpbk hm271qws rl04r1d5 l9j0dhe7 ov9facns kavbgo14')
        if raw_like_info == None:
            num_like = 0
        else:
            raw_like = raw_like_info.find('div', 'oajrlxb2 gs1a9yip g5ia77u1 mtkw9kbi tlpljxtp qensuy8j ppp5ayq2 goun2846 ccm00jje s44p3ltw mk2mc5f4 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv nhd2j8a9 mg4g778l pfnyh3mw p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x tgvbjcpo hpfvmrgz jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso l9j0dhe7 i1ao9s8h esuyzwwr f1sip0of du4w35lb n00je7tq arfg74bv qs9ysxi8 k77z8yql pq6dq46d btwxx1t3 abiwlrkh p8dawk7l lzcic4wl').attrs['aria-label']
            num_like = int(raw_like[:-1].split(' ')[-1])
        # 최고에요 수 수집
        raw_best_info = raw_lsc_info.find('span', 'np69z8it et4y5ytx j7g94pet b74d5cxt qw6c0r16 kb8x4rkr ed597pkb omcyoz59 goun2846 ccm00jje s44p3ltw mk2mc5f4 qxh1up0x qtyiw8t4 tpcyxxvw k0bpgpbk hm271qws rl04r1d5 l9j0dhe7 ov9facns tkr6xdv7')
        if raw_best_info == None:
            num_best = 0
        else:
            raw_best = raw_best_info.find('div', 'oajrlxb2 gs1a9yip g5ia77u1 mtkw9kbi tlpljxtp qensuy8j ppp5ayq2 goun2846 ccm00jje s44p3ltw mk2mc5f4 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv nhd2j8a9 mg4g778l pfnyh3mw p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x tgvbjcpo hpfvmrgz jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso l9j0dhe7 i1ao9s8h esuyzwwr f1sip0of du4w35lb n00je7tq arfg74bv qs9ysxi8 k77z8yql pq6dq46d btwxx1t3 abiwlrkh p8dawk7l lzcic4wl').attrs['aria-label']
            num_best = int(raw_best[:-1].split(' ')[-1])
        # 최종 좋아요 수
        like = num_like + num_best
        num_like_list.append(like)

print('좋아요 수 리스트 원소수 : ',len(num_like_list))

좋아요 수 리스트 원소수 :  7


In [141]:
num_like_list

[0, 1, 0, 1, 0, 0, 1]

In [57]:
from selenium.webdriver.common.action_chains import ActionChains

In [63]:
like_img = driver.find_element_by_xpath('//*[@id="jsc_c_2b"]/span[1]/span/span/div/img')

act = ActionChains(driver)

act.move_to_element(like_img).perform()

<ipython-input-63-1b0928f2a80e>:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  like_img = driver.find_element_by_xpath('//*[@id="jsc_c_2b"]/span[1]/span/span/div/img')


## 공유, 댓글 수 수집

In [177]:
# 공유 수, 댓글 수 수집
num_share_list = []
num_comment_list = []
for one in div:
    sc_info = one.find('div', 'bp9cbjyn j83agx80 pfnyh3mw p1ueia1e')
    if sc_info == None:
        num_share = 0
        num_comment = 0
    else:
        sc_list = sc_info.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
        if (sc_list == None) or ( len(sc_list) == 0 ):
            num_share = 0
            num_comment = 0
        else:
            for two in sc_list:
                two_str = two.text
                if '공유' in two_str:
                    try:
                        num_share = int(two_str[:-1].split(' ')[-1])
                    except:
                        num_share = two_str
                elif '댓글' in two_str:
                    try:
                        num_comment = int(two_str[:-1].split(' ')[-1])
                    except:
                        num_comment = two_str

    num_share_list.append(num_share)
    num_comment_list.append(num_comment)

print("공유 수의 개수 : ", len(num_share_list))
print("댓글 수의 개수 : ", len(num_comment_list))

공유 수의 개수 :  7
댓글 수의 개수 :  7


In [178]:
num_share_list

[0, 0, 0, 0, 0, 0, 1]

In [179]:
num_comment_list

[0, 0, 0, 0, 0, 0, 0]

## 게시글 링크 수집

### 로그인 안했을 때

In [180]:
# 게시글 링크 수집1(로그인❌)
link_list = []
for one in div:
    raw_link_info = one.find_all('div', 'qzhwtbm6 knvmm38d')[1]
    link = raw_link_info.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('?')[0]
    link_list.append(link)

print('링크 리스트 원소수 : ',len(link_list))

링크 리스트 원소수 :  7


In [181]:
link_list[:5]

['https://www.facebook.com/permalink.php',
 'https://www.facebook.com/permalink.php',
 'https://www.facebook.com/permalink.php',
 'https://www.facebook.com/permalink.php',
 'https://www.facebook.com/permalink.php']

### 로그인 했을 때

In [182]:
# 게시글 링크 수집2(로그인⭕)
link_list = []
for one in div:
    raw_link_info = one.find_all('div', 'qzhwtbm6 knvmm38d')[1]
    link = raw_link_info.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('cft')[0][:-3]
    link_list.append(link)
    
print('링크 리스트 원소수 : ',len(link_list))

링크 리스트 원소수 :  7


In [183]:
link_list[:5]

['https://www.facebook.com/permalink.php?story_fbid=108624708465224&id=100079530857939',
 'https://www.facebook.com/permalink.php?story_fbid=107093161951712&id=100079530857939',
 'https://www.facebook.com/permalink.php?story_fbid=106191052041923&id=100079530857939',
 'https://www.facebook.com/permalink.php?story_fbid=106171288710566&id=100079530857939',
 'https://www.facebook.com/permalink.php?story_fbid=106170925377269&id=100079530857939']

## 데이터 셋 만들기

In [184]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':num_date_list, '제목':title_list, '조회수':num_view_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-27,어제 시험 감독관 일이 생각보다 힘들었다. 그곳에서만 1만보를 넘게 걸었다.발목도 ...,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
1,2022-03-25,오늘은 페이스북 날짜 크롤링 코드를 조금 손봤다.항상 수집한 날짜가 다른 데이터보다...,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
2,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
3,2022-03-24,내 페이지의 정보를 내가 수집하면 차단되나?,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
4,2022-03-24,어떻게 내 페이지를 만드는 거지?어렵다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
5,2022-03-24,이렇게 하는게 맞는 건가?잘 모르겠다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
6,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,1,1,0,https://www.facebook.com/permalink.php?story_f...


## 데이터 셋. 날짜로 슬라이싱

In [185]:
data2 = data[data['날짜'] >= '2022-01-01']
data2

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-27,어제 시험 감독관 일이 생각보다 힘들었다. 그곳에서만 1만보를 넘게 걸었다.발목도 ...,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
1,2022-03-25,오늘은 페이스북 날짜 크롤링 코드를 조금 손봤다.항상 수집한 날짜가 다른 데이터보다...,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
2,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
3,2022-03-24,내 페이지의 정보를 내가 수집하면 차단되나?,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
4,2022-03-24,어떻게 내 페이지를 만드는 거지?어렵다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
5,2022-03-24,이렇게 하는게 맞는 건가?잘 모르겠다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
6,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,1,1,0,https://www.facebook.com/permalink.php?story_f...


## 필요한 데이터만 추출

In [187]:
# 작성 게시글만 추출
removed_words = ["Catch this week's", "["]

# 제거할 단어 들어있는 행 인덱스 찾기
temp = data2['제목'].to_list()
remove_index = []
for i, s in enumerate(temp):
    for one in removed_words:
        if one in s:
            remove_index.append(i)
remove_index

[]

In [188]:
# 중복 인덱스 제거
temp_idx_set = set(remove_index)
mod_remove_index = list(temp_idx_set)
mod_remove_index

[]

## 데이터 셋. 인덱스 내림차순으로 정렬

In [189]:
# 인덱스 리셋, 내림차순 정렬
data3 = data2.drop(mod_remove_index).reset_index(drop=True)
final_data = data3.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
6,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,1,1,0,https://www.facebook.com/permalink.php?story_f...
5,2022-03-24,이렇게 하는게 맞는 건가?잘 모르겠다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
4,2022-03-24,어떻게 내 페이지를 만드는 거지?어렵다,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
3,2022-03-24,내 페이지의 정보를 내가 수집하면 차단되나?,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
2,2022-03-24,오늘 처음 페이스북 계정을 만들었다.낯설지만 조금씩 작성해 보자,0,0,0,0,https://www.facebook.com/permalink.php?story_f...
1,2022-03-25,오늘은 페이스북 날짜 크롤링 코드를 조금 손봤다.항상 수집한 날짜가 다른 데이터보다...,0,1,0,0,https://www.facebook.com/permalink.php?story_f...
0,2022-03-27,어제 시험 감독관 일이 생각보다 힘들었다. 그곳에서만 1만보를 넘게 걸었다.발목도 ...,0,0,0,0,https://www.facebook.com/permalink.php?story_f...


## 엑셀 파일로 출력

In [190]:
# 수집한 데이터 파일료 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = './crawling_data/facebook/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

facebook_20220327-110848.xlsx 파일 생성 완료


In [ ]:
# chromedriver 종료
driver.quit()

# 코드 마지막